In [1]:
# allows to import own functions
import sys
import os
import re
root_project = re.findall(r'(^\S*TFM-master)', os.getcwd())[0]
sys.path.append(root_project)

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import json
import datetime
import seaborn as sns
import csv
sns.set()
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras import initializers

from src.utils.help_func import plot_predictions,\
    errors_distribution, get_model_data

import time

/home/agm/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6800202089033500371,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4871407476419389202
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6406922592
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3456923341692542504
 physical_device_desc: "device: 0, name: GeForce RTX 2060 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13902868350138324153
 physical_device_desc: "device: XLA_GPU device"]

In [4]:
df_train_val = get_model_data()
df_train_val

,i_country,R0,Tc,Tr,omega,inf_pow_1,inf_pow_2,mort_pow_1,mort_pow_2,mort_pow_3,...,degree,closeness,country_pop,country_departures,exposed_pop,inf_pow_1_log,inf_pow_2_log,mort_pow_1_log,mort_pow_2_log,mort_pow_3_log
0,UKR,6.468908,3.202028,20.713624,0.302601,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.306306,0.497758,44622516.0,76193,1.199770e+09,-36.043653,-36.043653,-36.043653,-36.043653,-36.043653
1,SWZ,2.027202,12.389972,25.116972,0.409112,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.009009,0.347418,1136191.0,4704,5.777962e+07,-36.043653,-36.043653,-36.043653,-36.043653,-36.043653
2,ZWE,17.968781,1.292124,23.217895,0.016608,10845.285714,1881.976378,0.021484,0.013648,0.224749,...,0.072072,0.405109,14439018.0,8183,3.124648e+08,9.291486,7.540078,-3.840447,-4.294169,-1.492773
3,NER,18.501724,1.117382,20.673491,0.967359,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.076577,0.417293,22442948.0,27,4.970584e+08,-36.043653,-36.043653,-36.043653,-36.043653,-36.043653
4,KIR,18.277633,1.609576,29.419238,0.665466,0.214286,-1.000000,1.333333,-4.000000,0.000000,...,0.036036,0.391534,115847.0,2566,3.281223e+08,-1.540445,NaN,0.287682,NaN,-36.043653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502666,KWT,11.989134,2.350649,28.182245,0.132076,21.714286,26.333333,0.299342,0.383966,2.843750,...,0.234234,0.484716,4137309.0,7258,2.959817e+09,3.077970,3.270836,-1.206168,-0.957201,1.045124
502667,ERI,13.971112,1.512712,21.134266,0.044862,1057.071429,554.055556,0.067572,0.050135,0.600601,...,0.049550,0.368159,3213972.0,745,2.538168e+08,6.963258,6.317265,-2.694560,-2.993029,-0.509825
502668,CHL,10.940605,2.647747,28.967954,0.013892,410.142857,52.880952,0.035876,0.030917,0.295129,...,0.175676,0.463466,18729160.0,10478,1.052402e+09,6.016506,3.968043,-3.327687,-3.476449,-1.220343
502669,MLI,15.535305,0.717363,11.144459,0.078483,138908.142857,65679.444444,0.058184,0.031904,0.401086,...,0.135135,0.417293,19077690.0,733,6.204510e+08,11.841568,11.092541,-2.844138,-3.445038,-0.913579


In [5]:
df_train_val.describe()

,R0,Tc,Tr,omega,inf_pow_1,inf_pow_2,mort_pow_1,mort_pow_2,mort_pow_3,n_closed,...,degree,closeness,country_pop,country_departures,exposed_pop,inf_pow_1_log,inf_pow_2_log,mort_pow_1_log,mort_pow_2_log,mort_pow_3_log
count,502671.000000,502671.000000,502671.000000,502671.000000,5.026710e+05,5.026710e+05,502671.000000,5.026710e+05,502671.000000,502671.000000,...,502671.000000,502671.000000,5.026710e+05,502671.000000,5.026710e+05,502671.000000,497601.000000,502671.000000,497599.000000,502671.000000
mean,13.365797,1.708106,17.230906,0.303155,2.096663e+06,3.965912e+05,0.219337,-2.128794e+00,1.630455,8.561441,...,0.192651,0.439393,3.697422e+07,19026.501191,1.581850e+09,-0.710729,-1.478249,-8.630257,-8.860472,-7.095957
std,5.736570,1.593771,7.669721,0.294660,2.006035e+07,3.063197e+06,2.512241,2.924663e+03,2.884641,5.953339,...,0.188582,0.065734,1.412452e+08,47300.766883,1.677997e+09,17.736539,17.714781,13.474482,13.770302,14.240566
min,2.000012,0.088688,2.000113,0.010000,0.000000e+00,-9.590619e+06,0.000000,-2.029014e+06,0.000000,0.000000,...,0.009009,0.296395,1.164600e+04,6.000000,2.425670e+05,-36.043653,-36.043653,-36.043653,-36.043653,-36.043653
25%,9.040197,0.813349,11.301047,0.038415,3.142857e+00,3.000000e+00,0.021222,1.154937e-02,0.149134,3.000000,...,0.063063,0.392920,1.136191e+06,1408.000000,3.924923e+08,1.145132,1.321756,-3.852712,-4.390786,-1.902909
50%,13.598063,1.270094,17.534982,0.202830,6.261429e+02,3.753750e+02,0.065668,4.714403e-02,0.473270,8.000000,...,0.126126,0.427746,6.982084e+06,4211.000000,7.287879e+08,6.439579,5.998671,-2.723141,-3.024623,-0.748088
75%,17.835840,1.958347,23.765149,0.522898,2.213021e+04,8.228938e+03,0.327230,3.476689e-01,1.936802,14.000000,...,0.252252,0.477419,2.554982e+07,16050.000000,2.532563e+09,10.004699,9.052124,-1.117091,-1.036225,0.661038
max,24.999916,14.892612,29.999951,0.999996,5.240514e+08,9.597455e+07,1274.000000,3.967781e+05,115.000000,19.000000,...,0.995495,0.634286,1.392730e+09,410193.000000,6.534162e+09,20.077100,18.379594,7.149917,12.891132,4.744932


In [6]:
features = [
    'Tr',
    'inf_pow_1',
    'inf_pow_2',
    'mort_pow_1',
    'mort_pow_2',
    'mort_pow_3',
    'n_closed',
    'react_time',
    'total_deceased',
    'betweenness',
    'degree',
    'closeness',
    'country_pop',
    'country_departures',
    'exposed_pop',
    'inf_pow_1_log',
    'inf_pow_2_log',
    'mort_pow_1_log',
    'mort_pow_2_log',
    'mort_pow_3_log',
    ]

# features = [
#     'Tr',
#     'n_closed',
#     'react_time',
#     'total_deceased',
#     'betweenness',
#     'degree',
#     'closeness',
#     'country_pop',
#     'country_departures',
#     'exposed_pop',
#     'inf_pow_1_log',
#     'inf_pow_2_log',
#     'mort_pow_1_log',
#     'mort_pow_2_log',
#     'mort_pow_3_log',
#     ]

In [7]:
df_train_val = df_train_val[features]
print("=" * 20)
print(f"Train_validation size: {df_train_val.shape}")
print("=" * 20)

Train_validation size: (502671, 20)


In [8]:
X_train_val = df_train_val.drop('total_deceased', axis=1)
y_train_val = df_train_val['total_deceased']

X_train, X_val, y_train, y_val = train_test_split(X_train_val,
                                                  y_train_val,
                                                 random_state=42)


size_data = int(len(X_train) / 1000)
num_features= len(X_train.columns)

In [9]:
pipe = Pipeline([
                 ('imputer', SimpleImputer(strategy='median')),
                 ('scaler',  StandardScaler())
])
                 

X_train_scaled = pipe.fit_transform(X_train.astype(np.float64))
X_val_scaled = pipe.transform(X_val.astype(np.float64))

In [10]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [11]:
root_logdir_tensorboard = f"{root_project}/models/my_logs"
root_logdir_checkpoints = f"{root_project}/models/checkpoints"

In [ ]:
# dense_layers = [4, 6, 8]
# layer_sizes = [20,40,60,80]

dense_layers = [8, 6, 4]
layer_sizes = [80, 60, 40, 20]

# dense_layers = [5]
# layer_sizes = [90]

revision = '14'

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        time.sleep(10)
        run_time = time.strftime("run_%d_%m_%Y-%H_%M_%S")
        NAME = f"{dense_layer}-layers-{layer_size}-nodes-{size_data}k-samples-{num_features}-feat-rev{revision}-{run_time}"
        print(NAME)
        model = Sequential()
        inputs = Input(X_train_scaled.shape[1:])
        
        for l in range(dense_layer-1):
            model.add(Dense(layer_size, activation='selu', kernel_initializer='lecun_normal'))
        
        model.add(Dense(1))
        
        tensorboard_cb = keras.callbacks.TensorBoard(f"{root_logdir_tensorboard}/{NAME}")
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
        checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath=f"{root_logdir_checkpoints}/{NAME}",
                                                       save_best_only=True, verbose=1)

        # model.compile(optimizer=Nadam(), loss='mean_squared_error', metrics=['mean_absolute_error'])
        model.compile(optimizer=Nadam(), loss='mean_squared_error',
                      metrics=['mean_absolute_error', 'mean_absolute_percentage_error', coeff_determination])

        
        history = model.fit(X_train_scaled, y_train, epochs=500, verbose=0,
                            validation_data=(X_val_scaled, y_val),
                           callbacks=[tensorboard_cb, early_stopping_cb, checkpoint_cb])

        score_train = model.evaluate(X_train_scaled, y_train, verbose=0)
        score_test = model.evaluate(X_val_scaled, y_val, verbose=0)
  
        
        print('Train score:', score_train[0])
        print('Train MAE:', score_train[1])
        print('Train r-squared:', score_train[2])
        print('Test score:', score_test[0])
        print('Test MAE:', score_test[1])
        print('Test r-squared:', score_test[2])
        print('========')

8-layers-80-nodes-377k-samples-19-feat-rev14-run_03_09_2020-15_12_23

Epoch 00001: val_loss improved from inf to 395720694467395584.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-80-nodes-377k-samples-19-feat-rev14-run_03_09_2020-15_12_23
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-80-nodes-377k-samples-19-feat-rev14-run_03_09_2020-15_12_23/assets

Epoch 00002: val_loss improved from 395720694467395584.00000 to 375927423583125504.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-80-nodes-377k-samples-19-feat-rev14-run_03_09_2020-15_12_23
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-80-nodes-377k-samples-19-feat-rev14-run_03_09_2020-15_12_23/assets

Epoch 00003: val_loss improved from 375927423583125504.00000 to 317218141224566784.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-80-nodes-377k-sam

INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-15_35_10/assets

Epoch 00006: val_loss improved from 298642682547273728.00000 to 284871831985324032.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-15_35_10
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-15_35_10/assets

Epoch 00007: val_loss did not improve from 284871831985324032.00000

Epoch 00008: val_loss did not improve from 284871831985324032.00000
Train score: 2.8655449709268173e+17
Train MAE: 265079088.0
Train r-squared: 1252096514981888.0
Test score: 2.8487183198532403e+17
Test MAE: 264382256.0
Test r-squared: 1252356763156480.0
8-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-15_45_00

Epoch 00001: val_loss improved from inf to 48086752775739801

Train score: 1.9631165074623693e+17
Train MAE: 206290160.0
Train r-squared: 15226378387456.0
Test score: 1.998328195543204e+17
Test MAE: 206787248.0
Test r-squared: 17366421340160.0
8-layers-20-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_04_47

Epoch 00001: val_loss improved from inf to 573952594481774592.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-20-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_04_47
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-20-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_04_47/assets

Epoch 00002: val_loss improved from 573952594481774592.00000 to 481432814173028352.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-20-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_04_47
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/8-layers-20-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_04_47/assets



Epoch 00009: val_loss did not improve from 301278624235913216.00000

Epoch 00010: val_loss improved from 301278624235913216.00000 to 283870228432027648.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-layers-80-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_16_21
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-layers-80-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_16_21/assets

Epoch 00011: val_loss improved from 283870228432027648.00000 to 274029015247880192.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-layers-80-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_16_21
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-layers-80-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_16_21/assets

Epoch 00012: val_loss improved from 274029015247880192.00000 to 269858911241109504.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-lay

INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_34_06/assets

Epoch 00011: val_loss improved from 350247333084004352.00000 to 347699283606110208.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_34_06
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_34_06/assets

Epoch 00012: val_loss did not improve from 347699283606110208.00000

Epoch 00013: val_loss improved from 347699283606110208.00000 to 337570410692345856.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_34_06
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/6-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-16_34_06/assets



...

INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-19_18_14/assets

Epoch 00081: val_loss did not improve from 326940126116577280.00000

Epoch 00082: val_loss improved from 326940126116577280.00000 to 325873943435018240.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-19_18_14
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-19_18_14/assets

Epoch 00083: val_loss improved from 325873943435018240.00000 to 324559064967151616.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-19_18_14
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-60-nodes-377k-samples-19-feat-rev14-run_03_09_2020-19_18_14/assets




Epoch 00019: val_loss improved from 475616122784186368.00000 to 471375890551406592.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00020: val_loss improved from 471375890551406592.00000 to 468370581675311104.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00021: val_loss improved from 468370581675311104.00000 to 465502436874780672.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:

INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00041: val_loss improved from 424452720007577600.00000 to 422721951266504704.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00042: val_loss improved from 422721951266504704.00000 to 421027088452026368.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00043: val_loss improved from 421027088452026368.00000 to 41917


Epoch 00062: val_loss improved from 385371678709907456.00000 to 383247731482689536.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00063: val_loss improved from 383247731482689536.00000 to 381004040567259136.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00064: val_loss improved from 381004040567259136.00000 to 379256779151769600.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:

INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00084: val_loss improved from 353002537424519168.00000 to 352782566379487232.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00085: val_loss improved from 352782566379487232.00000 to 350936383277236224.00000, saving model to /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41
INFO:tensorflow:Assets written to: /disks/hdd2T/Drive/TFM-master/models/checkpoints/4-layers-40-nodes-377k-samples-19-feat-rev14-run_03_09_2020-20_28_41/assets

Epoch 00086: val_loss improved from 350936383277236224.00000 to 35003